# 랜덤 포레스트 p.216
배깅 보팅, 여러 DecisionTree 객체로 보팅

In [1]:
# 중복된 피처명에 _1, _2 등 추가, p.203
def get_new_feature_name_df(old):
    import pandas as pd
    dup = pd.DataFrame(data=old.groupby('column_name').cumcount(), columns=['dup_cnt'])
    dup = dup.reset_index()
    new = pd.merge(old.reset_index(), dup, how='outer')
    # column_name과 dup_cnt 문자열을 합친다. : 'column_name'_'dup_cnt'
    new['column_name'] = new[['column_name', 'dup_cnt']].apply(lambda x: x[0]+'_'+str(x[1])
                                                               if x[1] > 0 else x[0], axis=1)
                                                              # dup_cnt값이 0보다 클 때만 합친다.
                                                              # 컬럼 단위로 처리하기 때문에 axis=1
    new = new.drop(['index'], axis=1)
    return new

In [2]:
def get_human_dataset():
    import pandas as pd
    # 각 데이터 파일은 공백으로 분리돼있음
    feature_name_df = pd.read_csv('human_activiry/UCI HAR Dataset/features.txt',
                              sep='\s+', # 분리기준 - '\s': 공백문자, '+': 1개 이상
                              header=None, # 헤어 이름 설정
                              names=['column_index', 'column_name'])
    
    # 중복된 피처명 수정, 신규 피처명 데이터프레임 생성
    new_name = get_new_feature_name_df(feature_name_df)
    
    # 데이터프레임에 피처명을 컬럼으로 부여하기위해 리스트 객체로 변환
    feature_name = new_name.iloc[:,1].values.tolist()
    
    # 학습 피처와 테스트 피처 데이터 로딩, 컬럼명 : feature_name 변수
    X_train = pd.read_csv('./human_activiry/UCI HAR Dataset/train/X_train.txt',sep='\s+',names=feature_name)
    X_test = pd.read_csv('./human_activiry/UCI HAR Dataset/test/X_test.txt',sep='\s+',names=feature_name)
    
    # 학습 레이블과 테스트 레이블 데이터 로딩, 컬럼명 : action 컬럼
    y_train = pd.read_csv('./human_activiry/UCI HAR Dataset/train/y_train.txt',sep='\s+',names=['action'])
    y_test = pd.read_csv('./human_activiry/UCI HAR Dataset/test/y_test.txt',sep='\s+',names=['action'])
    
    # 로드된 학습, 테스트용 데이터프레임 모두 반환
    return X_train, X_test, y_train, y_test

In [3]:
# 결정 트리에서 사용한 get_human_dataset()을 이용해 학습, 테스트용 데이터프레임 반환
X_train, X_test, y_train, y_test = get_human_dataset()

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [6]:
# 랜덤 포레스트 학습, 별도의 테스트 세트로 예측 성능 평가
# 랜덤포레스트 객체
rf_clf = RandomForestClassifier(random_state=0, max_depth=8) 
# max_depth를 조절해 과적합을 예방할 수 있다.
rf_clf.fit(X_train, y_train) # 학습
pred = rf_clf.predict(X_test) # 예측
round(accuracy_score(y_test, pred), 4) # 정확도 평가

C:\Users\user\AppData\Local\Temp\ipykernel_13756\863004386.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_clf.fit(X_train, y_train) # 학습


0.9196

In [ ]:
# 랜덤 포레스트 하이퍼 파라미터 튜닝, p.218

In [10]:
%%time
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')

params = {'max_depth': [8, 16, 24],
          'min_samples_leaf': [1, 6, 12],
          'min_samples_split': [2, 8, 16]}

# RandomForestClassifier 객체 생성 후 GridSearchCV 수행
rf_clf = RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=-1) # n_job : cpu 얼마나 쓸 거냐
grid_cv = GridSearchCV(rf_clf, param_grid=params, cv=2, n_jobs=-1) # 다 쓴다.
grid_cv.fit(X_train, y_train) # 학습

print(f'최적 하이퍼 파라미터: {grid_cv.best_params_}')
print(f'최고 예측 정확도: {grid_cv.best_score_:.4f}')

최적 하이퍼 파라미터: {'max_depth': 16, 'min_samples_leaf': 6, 'min_samples_split': 2}
최고 예측 정확도: 0.9165
CPU times: total: 13 s
Wall time: 37.8 s


In [11]:
# 최적의 하이퍼 파라미터로 다시
pred = grid_cv.best_estimator_.predict(X_test)
accuracy_score(y_test, pred)

0.9260264675941635

In [13]:
# 최적의 하이퍼 파라미터로 랜덤포레스트
rf_clf = RandomForestClassifier(n_estimators=500,
                                random_state=0,
                                n_jobs=-1,
                                max_depth=16,
                                min_samples_leaf=6,
                                min_samples_split=2)
rf_clf.fit(X_train, y_train)
pred = rf_clf.predict(X_test)
accuracy_score(y_test, pred)

0.9260264675941635

# GBM, p.221

In [15]:
from sklearn.ensemble import GradientBoostingClassifier

In [16]:
%%time

gb_clf = GradientBoostingClassifier(random_state=0)
gb_clf.fit(X_train, y_train)
gb_pred = gb_clf.predict(X_test)
accuracy_score(y_test, gb_pred)

CPU times: total: 15min 25s
Wall time: 15min 25s


0.9389209365456397